# フィルタ回路動作確認プログラム
- JPEG画像に対してフィルタ処理を実行し、表示回路2で結果を表示する
    - このプログラムと一緒に次の２つのJPEG画像ファイルをUploadしておくこと
    - image_vga.jpg
    - image_xga.jpg

### [STEP-1] 定数定義
- 表示解像度パラメータ（VGA or XGA）
- レジスタアドレス設定

In [ ]:
# 表示解像度パラメータ
PIXEL_VGA_WIDTH  = 640
PIXEL_VGA_HEIGHT = 480

PIXEL_XGA_WIDTH  = 1024
PIXEL_XGA_HEIGHT = 768

# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

# フィルタ回路レジスタオフセット
RA_FLTCTRL     = 0x00004000
RA_FLTSTAT     = 0x00004004
RA_FLTINT      = 0x00004008
RA_FLTVRAM_SRC = 0x0000400C
RA_FLTVRAM_FRM = 0x00004010
RA_FLTCOLOR    = 0x00004014

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

### [STEP-2] DisplayPort設定用クラス定義
- Zynq内部の専用レジスタの定義とレジスタ設定

In [ ]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

### [STEP-3] Overlay
- 表示回路１のコンフィギュレーション

In [ ]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-4] パターン表示回路の解像度設定
- RESOLの値を変更してパターン表示回路の解像度を設定
    - 0 : VGA設定
    - 1 : XGA設定
    - 2 : SXGA設定
- 表示回路２では、RESOLの値は0,1のみ対応

In [ ]:
RESOL = 0
pl.axi_gpio_0.write(RA_GPIO_DATA,(RESOL&1))
pl.axi_gpio_0.read(RA_GPIO_DATA)

0

### [STEP-5] DisplayPort初期化
- パターン表示回路の解像度に設定を合わせる
    - VGA設定  : dp.configure(VideoMode(640,480,24),PIXEL_RGB)
    - XGA設定  : dp.configure(VideoMode(1024,768,24),PIXEL_RGB)
    - SXGA設定 : dp.configure(VideoMode(1280,1024,24),PIXEL_RGB)

In [ ]:
from pynq.lib.video import *
lvdp = LiveDisplayPort()

if (RESOL & 1) == 1 :
    PIXEL_WIDTH  = PIXEL_XGA_WIDTH
    PIXEL_HEIGHT = PIXEL_XGA_HEIGHT
    FILE_JPG = 'image_xga.jpg'
    print("XGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)
else :
    PIXEL_WIDTH  = PIXEL_VGA_WIDTH
    PIXEL_HEIGHT = PIXEL_VGA_HEIGHT
    FILE_JPG = 'image_vga.jpg'
    print("VGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)

lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

VGA setting @ DisplayPort 640 480


### [STEP-6] VRAM領域確保
- ソース画像用VRAM ( SRC_VRAM_0 )
- 表示画像用VRAM   ( FRM_VRAM_0 )
- ともに１フレーム分の領域を確保
- 高さ x 幅 x 4byte(RGBA)で１フレーム

In [ ]:
from pynq import allocate

SRC_VRAM_0 = allocate(shape = (PIXEL_HEIGHT, PIXEL_WIDTH, 4), dtype = "u1")
FRM_VRAM_0 = allocate(shape = (PIXEL_HEIGHT, PIXEL_WIDTH, 4), dtype = "u1")

### [STEP-7] ソース画像用VRAMにJPEGデータを書き込む
- データの並びは、[B-8bit, G-8bit, R-8bit, A-8bit]の順

In [ ]:
import cv2
import numpy as np

img = cv2.imread(FILE_JPG)

SRC_VRAM_0[:,:,:] = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

### [STEP-8] フィルタ回路／表示回路のレジスタ設定
※このセルのフィルタ処理パラメータを変更して繰り返し実行してみる

（フィルタ回路）
- ソース画像用VRAM先頭アドレス設定
- フレームバッファ先頭アドレス設定(フィルタ処理後の画像格納先）
- フィルタ処理パラメータ設定
- フィルタ処理開始
- フィルタ処理完了待ち

（表示回路）
- 同期処理（VBLANK待ち)
- 表示開始先頭アドレス設定 @ DISPADDR
- 表示ON @ DISPCTRL

In [ ]:
# フィルタ回路
pl.regbus_0.write(RA_FLTVRAM_SRC, SRC_VRAM_0.device_address)
pl.regbus_0.write(RA_FLTVRAM_FRM, FRM_VRAM_0.device_address)
pl.regbus_0.write(RA_FLTCOLOR   , 0x00000007)
pl.regbus_0.write(RA_FLTCTRL    , 0x00000001)
while pl.regbus_0.read(RA_FLTSTAT) != 0x00000001 :
    pass

# 表示回路
pl.regbus_0.write(RA_DISPCTRL, 0x00000003)
while pl.regbus_0.read(RA_DISPCTRL) != 0x00000003 :
    pass

pl.regbus_0.write(RA_DISPADDR, FRM_VRAM_0.device_address)
pl.regbus_0.write(RA_DISPCTRL, 0x00000001)

### [STEP-9] 表示レジスタの設定
- 表示OFF @ DISPCTRL

In [ ]:
pl.regbus_0.write(RA_DISPCTRL, 0x00000000)

### [STEP-10] PYNQの初期画面に戻す
- DisplayPortの制御は、PL側からPS側に切り替わります。

In [ ]:
lvdp.close()